In [1]:
import parselmouth
from parselmouth.praat import call
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def compute_lh_ratio(audio_file):
    sound = parselmouth.Sound(audio_file)
    
    spectrum = sound.to_spectrum()
    
    lh_ratio = spectrum.get_band_energy(low_band_floor=0,low_band_ceiling=4000, high_band_floor=4000, high_band_ceiling=10000)
    
    return lh_ratio

In [11]:
compute_lh_ratio(audio_file='audio_files_pre/VOW/1234568/1234568_0_VOW_1_pre.wav')

TypeError: get_band_energy(): incompatible function arguments. The following argument types are supported:
    1. (self: parselmouth.Spectrum, band_floor: Optional[float] = None, band_ceiling: Optional[float] = None) -> float
    2. (self: parselmouth.Spectrum, band: Tuple[Optional[float], Optional[float]] = (None, None)) -> float

Invoked with: <parselmouth.Spectrum object at 0x7f4614440cf0>, 0, 4000, 4000, 10000